In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from scipy.signal import find_peaks, savgol_filter
import seaborn as sns
import os
import random
import cv2 as cv
import matplotlib.patches as patches
import re

In [3]:
# Set random seed
random.seed(41)

# Set max pixels
Image.MAX_IMAGE_PIXELS = None

In [4]:
# Seaborn style
sns.set()
sns.set_theme(style="darkgrid")
coolwarm = sns.color_palette("coolwarm")
pastel = sns.color_palette("pastel")

In [5]:
BASE_PATH = '/kaggle/input/UBC-OCEAN'
TRAIN_IMG_PATH = BASE_PATH + "/train_images"
TRAIN_THUMB_PATH = BASE_PATH + "/train_thumbnails"
TRAIN_ANN_PATH = BASE_PATH + "/train.csv"
TEST_IMG_PATH = BASE_PATH + "/test_thumbnails"
TEST_ANN_PATH = BASE_PATH + "/test.csv"

In [6]:
# Train annotations overview
train_df = pd.read_csv(TRAIN_ANN_PATH)
train_df.head()

,image_id,label,image_width,image_height,is_tma
0,4,HGSC,23785,20008,False
1,66,LGSC,48871,48195,False
2,91,HGSC,3388,3388,True
3,281,LGSC,42309,15545,False
4,286,EC,37204,30020,False


In [7]:
def path_column(image_id, path="/kaggle/input/UBC-OCEAN/train_images/"):
    return path + str(image_id) + ".png"

train_df["file_path"] = train_df["image_id"].map(path_column)

In [8]:
import os
import gc
import cv2
import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, f1_score

import albumentations as A
from albumentations.pytorch import ToTensorV2

# For Image Models
import timm

# Albumentations for augmentations
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
# warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [9]:
CONFIG = {
    "is_submission": False,
    "n_fold": 5,
    'fold': 1,
    "seed": 42,
    "img_size": 512,
    "crop_vertical":True,
    "model_name": "tf_efficientnet_b0_ns",
    "num_classes": 5,
    "valid_batch_size": 16,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    # "model_path": '/kaggle/input/efficientnetb0-training-crop-images/best_model_checkpoint2023-10-26_09-10-29.pth',
    "encoder_path": "/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b2/1/tf_efficientnet_b2_aa-60c94f97.pth"
}

In [10]:
ROOT_DIR = '/kaggle/input/UBC-OCEAN'
TRAIN_DIR = '/kaggle/input/UBC-OCEAN/train_thumbnails'
TEST_DIR = '/kaggle/input/UBC-OCEAN/test_thumbnails'
ALT_TEST_DIR = '/kaggle/input/UBC-OCEAN/test_images'
TMA_TRAIN_DIR = '/kaggle/input/UBC-OCEAN/train_images'


def get_train_file_path(df_train_row):
    if df_train_row.is_tma == False:
        return f"{TRAIN_DIR}/{df_train_row.image_id}_thumbnail.png"
    else:
        return f"{TMA_TRAIN_DIR}/{df_train_row.image_id}.png"



def get_test_file_path(image_id):
    if os.path.exists(f"{TEST_DIR}/{image_id}_thumbnail.png"):
        return f"{TEST_DIR}/{image_id}_thumbnail.png"
    else:
        return f"{ALT_TEST_DIR}/{image_id}.png"


In [11]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if "best_model" in filename:
            print(dirname+"/"+filename)
        continue

In [12]:
df_test = pd.read_csv("/kaggle/input/UBC-OCEAN/test.csv")
df_test['file_path'] = df_test['image_id'].apply(get_test_file_path)
df_test["target_label"] = 0 
df_test

,image_id,image_width,image_height,file_path,target_label
0,41,28469,16987,/kaggle/input/UBC-OCEAN/test_thumbnails/41_thu...,0


In [13]:
class UBCDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.filenames = df.file_path.values
        self.labels =  df.target_label.values
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if CONFIG["crop_vertical"]:
            img = crop_vertical(img)
        
        # img = custom_center_crop_or_resize(img, (1024, 1024))
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            "image": img,
            "label": torch.tensor(self.labels[idx], dtype=torch.long)
               }
    
def crop_vertical(image):
    """
    Function crops images if multiple slices contained and separated by black vertical background.
    """
    vertical_sum = np.sum(image, axis=(0, 2))

    # Identify the positions where the sum is zero
    zero_positions = np.where(vertical_sum == 0)[0]

    if len(zero_positions)==0:
        cropped_images = [image]
    else:
        # If the image does not start with a black area, add index 0
        if zero_positions[0] != 0:
            zero_positions = np.insert(zero_positions, 0, 0)

        # If the image does not end with a black area, add the image width
        if zero_positions[-1] != image.shape[1] - 1:
            zero_positions = np.append(zero_positions, image.shape[1] - 1)

        start_idx = zero_positions[0]
        cropped_images = []

        for idx in range(1, len(zero_positions)):
            end_idx = zero_positions[idx]
            if end_idx - start_idx > 1:  # If the width of the cropped section is greater than 1
                cropped = image[:, start_idx:end_idx]
                # only include samples which are of min size
                if cropped.shape[1]>200:  
                    cropped_images.append(cropped)
                    # cv2.imwrite(f"{save_prefix}_{idx}.jpg", cropped)
            start_idx = end_idx
            

def crop_vertical(image):
    """
    Function crops images if multiple slices contained and separated by black vertical background.
    """
    vertical_sum = np.sum(image, axis=(0, 2))

    # Identify the positions where the sum is zero
    zero_positions = np.where(vertical_sum == 0)[0]

    if len(zero_positions)==0:
        cropped_images = [image]
    else:
        # If the image does not start with a black area, add index 0
        if zero_positions[0] != 0:
            zero_positions = np.insert(zero_positions, 0, 0)

        # If the image does not end with a black area, add the image width
        if zero_positions[-1] != image.shape[1] - 1:
            zero_positions = np.append(zero_positions, image.shape[1] - 1)

        start_idx = zero_positions[0]
        cropped_images = []

        for idx in range(1, len(zero_positions)):
            end_idx = zero_positions[idx]
            if end_idx - start_idx > 1:  # If the width of the cropped section is greater than 1
                cropped = image[:, start_idx:end_idx]
                # only include samples which are of min size
                if cropped.shape[1]>200:  
                    cropped_images.append(cropped)
                    # cv2.imwrite(f"{save_prefix}_{idx}.jpg", cropped)
            start_idx = end_idx
    final_crops = []
    # remove black bars above/below the crops 
    for cropped in cropped_images:
        horizontal_sum = np.sum(cropped, axis=(1, 2))
        zero_positions = np.where(horizontal_sum == 0)[0]
        img_ = np.delete(cropped, zero_positions, axis=0)
        final_crops.append(img_)
    if len(final_crops)==0:
        return image
    return final_crops[0]


def custom_center_crop_or_resize(image, crop_size):
    # If both dimensions of the image are greater than or equal to the desired size, apply CenterCrop
    if image.shape[0] >= crop_size[0] and image.shape[1] >= crop_size[1]:
        return A.CenterCrop(crop_size[0], crop_size[1])(image=image)["image"]
    # Else, just resize the image to the desired size
    else:
        return A.Resize(crop_size[0], crop_size[1])(image=image)["image"]

In [14]:
data_transforms = {
    "train": A.Compose([
        A.RandomResizedCrop(CONFIG['img_size'], CONFIG['img_size'], scale=(0.8, 1.0)),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.2),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        A.CoarseDropout(p=0.2),
        A.Cutout(p=0.2),
        A.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225], 
            max_pixel_value=255.0, 
            p=1.0
        ),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225], 
            max_pixel_value=255.0, 
            p=1.0
        ),
        ToTensorV2()], p=1.)
}

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


In [15]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'

class EfficientNetB0(nn.Module):
    '''
    EfficientNet B0 fine-tune.
    '''
    def __init__(self, model_name, num_classes, pretrained=True, checkpoint_path=None):
        '''
        Fine tune for EfficientNetB0
        Args
            n_classes : int - Number of classification categories.
            learnable_modules : tuple - Names of the modules to fine-tune.
        Return
            
        '''
        super(EfficientNetB0, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, checkpoint_path=checkpoint_path)

        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.pooling = GeM()
        self.linear = nn.Linear(in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)
    
    
    def forward(self, images):
        """
        Forward function for the fine-tuned model
        Args
            x: 
        Return
            result
        """
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        output = self.linear(pooled_features)
        return output

In [16]:
import torch.nn.functional as F


In [17]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [18]:
def predict_val_dataset(model, CONFIG, df_validate, TRAIN_DIR=None, val_size=1.0):
    if not CONFIG["is_submission"]:
        valid_dataset = UBCDataset(df_validate, transforms=data_transforms["valid"])
        valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)

        preds = []
        labels_list = []
        valid_acc = 0.0

        with torch.no_grad():
            bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
            for step, data in bar: 
                # print(step)
                images = data['image'].to(CONFIG["device"], dtype=torch.float)
                labels = data['label'].to(CONFIG["device"], dtype=torch.long)

                batch_size = images.size(0)
                outputs = model(images)
                print("outputs", outputs)
                probs = F.softmax(outputs, dim=1)
                print("probs", probs)
                _ , predicted = torch.max(probs, dim=1)
                print("predicted", predicted)
                preds.append( predicted.detach().cpu().numpy() )
                print("predicted.detach().cpu().numpy()", predicted.detach().cpu().numpy())
                labels_list.append( labels.detach().cpu().numpy() )
                acc = torch.sum( predicted == labels )
                valid_acc  += acc.item()
        valid_acc /= len(valid_loader.dataset)
        print("preds", preds)
        preds = np.concatenate([label_encoder.inverse_transform(batch) for batch in preds])
        labels_list = np.concatenate(labels_list).flatten()

        
        # Calculate Balanced Accuracy
        bal_acc = balanced_accuracy_score(labels_list, preds)
        # Calculate Confusion Matrix
        conf_matrix = confusion_matrix(labels_list, preds)
        macro_f1 = f1_score(labels_list, preds, average='macro')
        micro_f1 = f1_score(labels_list, preds, average='micro')
        weighted_f1 = f1_score(labels_list, preds, average='weighted')
    
        print(f"Validation Accuracy: {valid_acc}")
        print(f"Balanced Accuracy: {bal_acc}")
        print(f"Macro F1-Score: {macro_f1}")
        print(f"Micro F1-Score: {micro_f1}")
        print(f"Weighted F1-Score: {weighted_f1}")
        print(f"Confusion Matrix: {conf_matrix}")
        
        # add to validation dataframe
        df_validate["pred"] = preds
        df_validate["pred_labels"] = pred_labels
        return df_validate, preds, labels_list
    else:
        print("Skip validation on training set due to submission!")
        return None

In [19]:
import timm
model = timm.create_model('tf_efficientnet_b2', pretrained = True,checkpoint_path='/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b2/1/tf_efficientnet_b2_aa-60c94f97.pth')

In [20]:
if not CONFIG["is_submission"]: 
    df_train = pd.read_csv("/kaggle/input/UBC-OCEAN/train.csv")
    print(df_train.shape)
    df_train['file_path'] = df_train.apply(lambda row: get_train_file_path(row), axis=1)
    df_train['target_label'] = label_encoder.fit_transform(df_train['label'])
    # use stratified K Fold for crossvalidation 
    skf = StratifiedKFold(n_splits=CONFIG['n_fold'], shuffle=True, random_state=CONFIG["seed"])
    for fold, ( _, val_) in enumerate(skf.split(X=df_train, y=df_train.target_label)):
        df_train.loc[val_ , "kfold"] = int(fold)
        
    all_labels = []
    all_predictions = []
#     for fold, model in enumerate(models):
    fold = 1
    print("Evaluate Fold: ", fold)
    
    model.eval()
    model.to(CONFIG["device"])
    df_train_fold = df_train[df_train["kfold"]!=fold].reset_index(drop=True)
    df_valid_fold = df_train[df_train["kfold"]==fold].reset_index(drop=True)

    #train_dataset = UBCDataset(df_train_fold, transforms=data_transforms["train"])
    #train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
    #                          num_workers=2, shuffle=False, pin_memory=True)
    valid_dataset = UBCDataset(df_valid_fold, transforms=data_transforms["valid"])
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)

    df_validate, predictions, labels = predict_val_dataset(model, CONFIG, df_valid_fold, TRAIN_DIR, val_size=1)
    all_labels.extend(labels)
    all_predictions.extend(predictions)
    display(df_validate)

(538, 5)
Evaluate Fold:  1


 14%|█▍        | 1/7 [00:19<01:58, 19.74s/it]

outputs tensor([[-0.9517, -0.1615, -0.7614,  ...,  2.0117, -0.7561,  0.2888],
        [-1.5231, -0.7579,  0.4153,  ...,  1.1984, -0.8694,  0.1046],
        [-1.3980, -0.3548, -0.7353,  ...,  0.4304, -0.3972,  0.2442],
        ...,
        [-1.2291, -0.3650, -0.7254,  ..., -1.0482,  0.0654,  1.5473],
        [-1.3988, -0.9760, -1.2871,  ...,  1.3922, -0.0301,  2.1638],
        [-0.9439, -0.8932, -1.0316,  ...,  0.0856, -0.2255,  0.3678]])
probs tensor([[1.0263e-04, 2.2618e-04, 1.2415e-04,  ..., 1.9872e-03, 1.2480e-04,
         3.5483e-04],
        [7.0817e-05, 1.5221e-04, 4.9202e-04,  ..., 1.0767e-03, 1.3616e-04,
         3.6059e-04],
        [3.8944e-05, 1.1054e-04, 7.5552e-05,  ..., 2.4238e-04, 1.0594e-04,
         2.0121e-04],
        ...,
        [1.0297e-04, 2.4435e-04, 1.7041e-04,  ..., 1.2340e-04, 3.7576e-04,
         1.6538e-03],
        [5.3695e-05, 8.1949e-05, 6.0037e-05,  ..., 8.7511e-04, 2.1103e-04,
         1.8929e-03],
        [6.1885e-05, 6.5099e-05, 5.6686e-05,  ..., 1.7

 29%|██▊       | 2/7 [00:30<01:11, 14.27s/it]

outputs tensor([[-2.1068, -0.1590,  0.6800,  ..., -0.6296,  0.0458,  0.4443],
        [-1.2023, -0.1873, -0.6073,  ...,  1.5965,  0.4899,  0.1716],
        [-1.2637,  0.5375, -1.2529,  ...,  0.4702, -0.6430, -0.4114],
        ...,
        [-0.5984,  0.2889, -0.6824,  ..., -0.7873, -1.5014,  1.0161],
        [-0.7485, -0.2368, -1.0035,  ...,  0.4255, -0.8885,  2.0216],
        [-0.8583, -0.9881, -1.2434,  ...,  1.3039,  0.1728,  1.0047]])
probs tensor([[2.8463e-05, 1.9961e-04, 4.6191e-04,  ..., 1.2468e-04, 2.4498e-04,
         3.6494e-04],
        [4.6395e-05, 1.2802e-04, 8.4111e-05,  ..., 7.6205e-04, 2.5197e-04,
         1.8328e-04],
        [4.7359e-05, 2.8686e-04, 4.7878e-05,  ..., 2.6818e-04, 8.8106e-05,
         1.1106e-04],
        ...,
        [1.2347e-04, 2.9987e-04, 1.1353e-04,  ..., 1.0222e-04, 5.0050e-05,
         6.2049e-04],
        [8.0123e-05, 1.3365e-04, 6.2085e-05,  ..., 2.5918e-04, 6.9653e-05,
         1.2787e-03],
        [6.1342e-05, 5.3877e-05, 4.1737e-05,  ..., 5.3

...

 86%|████████▌ | 6/7 [01:06<00:09,  9.61s/it]

outputs tensor([[-1.6774, -0.6642,  0.2020,  ...,  0.9542, -0.5470,  0.7588],
        [-1.9593, -0.2336, -1.0884,  ..., -1.0418, -1.4322,  0.3138],
        [-1.2798,  0.3062, -1.2653,  ...,  0.7059, -0.9658, -0.0644],
        ...,
        [-0.9782, -0.6225, -1.0451,  ...,  0.5818, -0.3006,  1.1562],
        [-0.7867, -0.0089, -1.0390,  ...,  1.3397,  0.1183,  1.3414],
        [-1.3858, -0.2589,  0.2762,  ..., -0.1039, -0.6444,  1.2557]])
probs tensor([[2.4232e-05, 6.6740e-05, 1.5870e-04,  ..., 3.3671e-04, 7.5039e-05,
         2.7696e-04],
        [3.8467e-05, 2.1604e-04, 9.1898e-05,  ..., 9.6285e-05, 6.5164e-05,
         3.7347e-04],
        [5.8959e-05, 2.8797e-04, 5.9822e-05,  ..., 4.2946e-04, 8.0711e-05,
         1.9879e-04],
        ...,
        [8.8157e-05, 1.2583e-04, 8.2458e-05,  ..., 4.1952e-04, 1.7360e-04,
         7.4511e-04],
        [1.0991e-04, 2.3925e-04, 8.5405e-05,  ..., 9.2159e-04, 2.7170e-04,
         9.2308e-04],
        [3.7539e-05, 1.1585e-04, 1.9783e-04,  ..., 1.3

100%|██████████| 7/7 [01:12<00:00, 10.35s/it]

outputs tensor([[-1.5157, -0.5443, -0.5928,  ...,  0.8604, -0.4771,  1.0077],
        [-0.7544, -0.1305, -1.4716,  ...,  0.3403, -0.4963,  0.6633],
        [-0.4770, -0.7931, -0.7533,  ...,  1.8620,  0.3759,  0.4979],
        ...,
        [-0.5093, -0.4928, -0.1744,  ..., -0.3098,  0.5992,  1.0548],
        [-1.9054,  0.3613, -1.0241,  ...,  0.8766, -0.4535,  1.3802],
        [-1.4771, -0.5835, -0.3725,  ..., -0.3096, -0.3376,  0.2507]])
probs tensor([[3.9052e-05, 1.0316e-04, 9.8271e-05,  ..., 4.2028e-04, 1.1033e-04,
         4.8699e-04],
        [1.2418e-04, 2.3175e-04, 6.0620e-05,  ..., 3.7111e-04, 1.6076e-04,
         5.1257e-04],
        [1.0660e-04, 7.7708e-05, 8.0856e-05,  ..., 1.1055e-03, 2.5012e-04,
         2.8257e-04],
        ...,
        [1.5670e-04, 1.5932e-04, 2.1905e-04,  ..., 1.9131e-04, 4.7479e-04,
         7.4882e-04],
        [3.5003e-05, 3.3770e-04, 8.4498e-05,  ..., 5.6535e-04, 1.4951e-04,
         9.3547e-04],
        [3.9557e-05, 9.6674e-05, 1.1938e-04,  ..., 1.2

ValueError: y contains previously unseen labels: [111 529 591 649 658 793 824 885 911 936 938 980]